<a href="https://colab.research.google.com/github/nof21/random-tutorials/blob/main/carbon_allocation_ghg_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Closed Loop vs. Recycled Content**

The **Closed Loop** and **Recycled Content** allocate emissions and removals differently, and choosing one method over the other can produce different inventory results.

This simplified example highlights the difference: in this case, both methods are equally appropriate for a product with **virgin and recycled material input, recycled material output, and waste**.

---

## **Example Parameters**

| **Data**                                   | **Value** | **Units**                         |
|--------------------------------------------|----------|----------------------------------|
| **Material Input**                         | 5        | tons                             |
| **Material Output**                        | 5        | tons                             |
| **Recycled Material Input**                | 40%      | Percent of total input          |
| **Virgin Material Input**                  | 60%      | Percent of total input          |
| **Recycled Material Output**               | 25%      | Percent of total output         |
| **Waste Output**                           | 75%      | Percent of total output         |
| **Virgin Material Acquisition & Preprocessing** | 10   | kg CO₂e/ton                     |
| **Recycled Material Acquisition (MRF) & Preprocessing** | 3   | kg CO₂e/ton                     |
| **Waste Treatment**                        | 5        | kg CO₂e/ton                     |

---

This exercise allows you to explore how **changing the recycled content and output percentages** affects total emissions under **both allocation methods**. The interactive plots compare the two approaches, helping you visualize the differences in carbon footprints under different recycling scenarios.

Based on the example available in the GHG Protocol Product Life Cycle Accounting and Reporting Standard.


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

plt.rcParams['font.family'] ='sans-serif' #sans-serif
plt.rcParams['font.size']=18

#**Define constants**

Be aware that modifying these values from the original description, might result in different values

In [2]:
input_tons = 5  # Total input in tons
output_tons = 5  # Total final output in tons
waste_treatment_emission = 5  # kg CO2e per ton for waste treatment
virgin_acq_emission = 10  # kg CO2e per ton for virgin material
recycled_acq_emission = 3  # kg CO2e per ton for recycled material

categories = ['Material Acquisition', 'End of Life', 'Material Displacement', 'Total']

# Comparison of Recycled Content vs. Closed Loop Approaches

This code calculates and compares the total emissions (kg CO₂e) for two different material allocation methods:  
1. **Recycled Content Approach:** Allocates impacts based on the share of recycled and virgin materials in the input.  
2. **Closed Loop Approach:** Accounts for material displacement, considering how recycled output reduces the need for virgin material.  



In [5]:
# Function to update plot dynamically
def update_plot(recycled_input_pct, recycled_output_pct):
    # Convert percentage inputs to fractions
    recycled_input_pct /= 100  # Convert from 0-100 to 0-1
    recycled_output_pct /= 100  # Convert from 0-100 to 0-1
    virgin_input_pct = 1 - recycled_input_pct
    waste_output_pct = 1 - recycled_output_pct  # Waste fraction

    # Recycled approach calculations
    rec_mat_acq = (input_tons * recycled_input_pct * recycled_acq_emission +
                   input_tons * virgin_input_pct * virgin_acq_emission)
    rec_end = output_tons * waste_output_pct * waste_treatment_emission
    rec_total = rec_mat_acq + rec_end

    # Closed-loop approach calculations
    cl_mat_acq = input_tons * virgin_acq_emission
    cl_end = (output_tons * waste_output_pct * waste_treatment_emission +
              output_tons * recycled_output_pct * recycled_acq_emission)
    cl_virgin_displacement = output_tons * recycled_output_pct * virgin_acq_emission
    cl_total = cl_mat_acq + cl_end - cl_virgin_displacement

    # Updated values including Material Displacement and Total
    rec_values = [rec_mat_acq, rec_end, 0, rec_total]
    cl_values = [cl_mat_acq, cl_end, -cl_virgin_displacement, cl_total]

    # Create plot
    fig, ax = plt.subplots(figsize=(16, 8))
    y = np.arange(len(categories))
    width = 0.4

    rects1 = ax.barh(y - width/2, rec_values, width,color='#053061', label='Recycled Content')
    rects2 = ax.barh(y + width/2, cl_values, width,color = '#bababa', label='Closed Loop ')

    # Labels and title
    ax.set_xlabel('Emissions (kg CO$_2$e)',weight='bold')
    ax.set_title(f'Comparison of Emissions:\n Recycled Content vs Closed Loop\n'
                 f'Recycled Input: {recycled_input_pct*100:.0f}%\n Recycled Output: {recycled_output_pct*100:.0f}%',weight='bold')
    ax.set_yticks(y)
    ax.set_yticklabels(categories,weight='bold')
    ax.legend(frameon=False)

    # Show values on bars
    for rects in [rects1, rects2]:
        for rect in rects:
            width = rect.get_width()
            ax.annotate(f'{width:.0f}',
                        xy=(width, rect.get_y() + rect.get_height() / 2),
                        xytext=(3, 0),
                        textcoords="offset points",
                        weight = 'bold',
                        ha='left' if width >= 0 else 'right', va='center')

    plt.show()

# Apply Arial font style using HTML
style_html = """
<style>
    .widget-label {
        font-family: Arial, sans-serif !important;
        font-size: 18px;
    }
</style>
"""

# Display the style to change font globally for ipywidgets labels
display(widgets.HTML(style_html))

# Interactive numeric input boxes with increased width
interact(update_plot,
         recycled_input_pct=widgets.BoundedFloatText(
             value=40, min=0, max=100, step=5,
             description="Recycled Input %",
             style={'description_width': 'auto'},
             layout=widgets.Layout(width="200px")),

         recycled_output_pct=widgets.BoundedFloatText(
             value=25, min=0, max=100, step=5,
             description="Recycled Output %",
             style={'description_width': 'auto'},
             layout=widgets.Layout(width="210px")));


HTML(value='\n<style>\n    .widget-label {\n        font-family: Arial, sans-serif !important;\n        font-s…

interactive(children=(BoundedFloatText(value=40.0, description='Recycled Input %', layout=Layout(width='200px'…

# Sensitivity Analysis

Impact of Recycled Input and Output on Emissions for Two Approaches (Recycled & Closed Loop).

- The code iterates over recycling input (%) and output (%) to analyze their influence on total emissions.
- Figure 1: Varies recycling input (%), keeping recycling output fixed.
- Figure 2: Varies recycling output (%), keeping recycling input fixed.
- Key insight: Shows how changes in material flow affect emissions in Recycled Content and Closed Loop approaches.


In [6]:
# Define ranges for iteration
recycled_input_range = np.linspace(0, 100, 21)  # From 0 to 100, step of 5%
recycled_output_range = np.linspace(0, 100, 21)  # From 0 to 100, step of 5%

# Function to update the plots dynamically based on user input
def update_plots(fixed_recycled_output, fixed_recycled_input):
    # Store results
    rec_total_input_var = []
    cl_total_input_var = []
    rec_total_output_var = []
    cl_total_output_var = []

    # Iterate over recycled input percentage while keeping recycled output fixed
    for recycled_input_pct in recycled_input_range:
        recycled_input_pct /= 100  # Convert to fraction
        virgin_input_pct = 1 - recycled_input_pct
        waste_output_pct = 1 - (fixed_recycled_output / 100)  # Convert to fraction

        # Recycled approach calculations
        rec_mat_acq = (input_tons * recycled_input_pct * recycled_acq_emission +
                       input_tons * virgin_input_pct * virgin_acq_emission)
        rec_end = output_tons * waste_output_pct * waste_treatment_emission
        rec_total = rec_mat_acq + rec_end

        # Closed-loop approach calculations
        cl_mat_acq = input_tons * virgin_acq_emission
        cl_end = (output_tons * waste_output_pct * waste_treatment_emission +
                  output_tons * (fixed_recycled_output / 100) * recycled_acq_emission)
        cl_virgin_displacement = output_tons * (fixed_recycled_output / 100) * virgin_acq_emission
        cl_total = cl_mat_acq + cl_end - cl_virgin_displacement

        rec_total_input_var.append(rec_total)
        cl_total_input_var.append(cl_total)

    # Iterate over recycled output percentage while keeping recycled input fixed
    for recycled_output_pct in recycled_output_range:
        recycled_output_pct /= 100  # Convert to fraction
        virgin_input_pct = 1 - (fixed_recycled_input / 100)
        waste_output_pct = 1 - recycled_output_pct

        # Recycled approach calculations
        rec_mat_acq = (input_tons * (fixed_recycled_input / 100) * recycled_acq_emission +
                       input_tons * virgin_input_pct * virgin_acq_emission)
        rec_end = output_tons * waste_output_pct * waste_treatment_emission
        rec_total = rec_mat_acq + rec_end

        # Closed-loop approach calculations
        cl_mat_acq = input_tons * virgin_acq_emission
        cl_end = (output_tons * waste_output_pct * waste_treatment_emission +
                  output_tons * recycled_output_pct * recycled_acq_emission)
        cl_virgin_displacement = output_tons * recycled_output_pct * virgin_acq_emission
        cl_total = cl_mat_acq + cl_end - cl_virgin_displacement

        rec_total_output_var.append(rec_total)
        cl_total_output_var.append(cl_total)

    # Create subplots
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))

    # First subplot - Varying recycled input
    axes[0].plot(recycled_input_range, rec_total_input_var,color='#053061', label='Recycled Approach', marker='o',linewidth=3)
    axes[0].plot(recycled_input_range, cl_total_input_var,color = '#bababa', label='Closed Loop Approach', marker='s',linewidth=3)
    axes[0].set_xlabel("Recycled Input (%)",weight='bold')
    axes[0].set_ylabel("Total Emissions (kg CO$_2$e)",weight='bold')
    axes[0].set_title(f"Impact of Recycled Input (Fixed Output at {fixed_recycled_output}%)",weight='bold')
    # axes[0].legend(frameon = False)
    axes[0].grid(color='lightgray', linestyle='-', linewidth=0.5, alpha=0.25)

    # Second subplot - Varying recycled output
    axes[1].plot(recycled_output_range, rec_total_output_var,color='#053061', label='Recycled Content', marker='o',linewidth=3)
    axes[1].plot(recycled_output_range, cl_total_output_var,color = '#bababa', label='Closed Loop', marker='s',linewidth=3)
    axes[1].set_xlabel("Recycled Output (%)",weight='bold')
    axes[1].set_ylabel("Total Emissions (kg CO$_2$e)",weight='bold')
    axes[1].set_title(f"Impact of Recycled Output (Fixed Input at {fixed_recycled_input}%)",weight='bold')
    axes[1].legend(frameon = False)
    axes[1].grid(color='lightgray', linestyle='-', linewidth=0.5, alpha=0.25)

    plt.tight_layout()
    plt.show()

# Create interactive widgets
interact(update_plots,
         fixed_recycled_output=widgets.BoundedFloatText(value=25, min=0, max=100, step=5,style={'description_width': 'auto'}, description="Fixed Output %", layout=widgets.Layout(width="175px")),
         fixed_recycled_input=widgets.BoundedFloatText(value=25, min=0, max=100, step=5,style={'description_width': 'auto'}, description="Fixed Input %", layout=widgets.Layout(width="175px")));


interactive(children=(BoundedFloatText(value=25.0, description='Fixed Output %', layout=Layout(width='175px'),…